In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_path = "/content/drive/MyDrive/job_question_generator3"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()
if torch.cuda.is_available():
    model.to("cuda")

def generate_questions(job_title, num_questions=5):
    prompt = f"Role: {job_title}\nQuestion:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=50,
            num_return_sequences=num_questions,
            do_sample=True,
            temperature=0.8,
            top_p=0.9
        )

    questions = []
    for o in outputs:
        text = tokenizer.decode(o, skip_special_tokens=True)
        question_only = text.replace(f"Role: {job_title}\nQuestion:", "").strip()
        questions.append(question_only)

    return "\n".join(questions)

iface = gr.Interface(
    fn=generate_questions,
    inputs=[gr.Textbox(label="Job Title")],
    outputs=gr.Textbox(label="Generated Questions", lines=10),
    title="Interview Question Generator"
)

iface.launch(share=True)